In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/email-spam/spam_dataset/dataset_dict.json
/kaggle/input/email-spam/spam_dataset/val/state.json
/kaggle/input/email-spam/spam_dataset/val/dataset_info.json
/kaggle/input/email-spam/spam_dataset/val/data-00000-of-00001.arrow
/kaggle/input/email-spam/spam_dataset/test/state.json
/kaggle/input/email-spam/spam_dataset/test/dataset_info.json
/kaggle/input/email-spam/spam_dataset/test/data-00000-of-00001.arrow
/kaggle/input/email-spam/spam_dataset/train/state.json
/kaggle/input/email-spam/spam_dataset/train/dataset_info.json
/kaggle/input/email-spam/spam_dataset/train/data-00000-of-00001.arrow


In [2]:
from datasets import load_from_disk

dataset = load_from_disk("/kaggle/input//email-spam/spam_dataset")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'Spam'],
        num_rows: 4556
    })
    val: Dataset({
        features: ['Text', 'Spam'],
        num_rows: 569
    })
    test: Dataset({
        features: ['Text', 'Spam'],
        num_rows: 570
    })
})

In [4]:
train = dataset["train"]
test = dataset["test"]
val = dataset["val"]

In [5]:
len(train)

4556

In [6]:
for entry in train.select(range(1)):
    sms = entry["Text"]
    label = entry["Spam"]
    print(f"label={label}, sms={sms}")

label=0, sms=Subject: aiesec polska - eurolds 2000  jarek ,  czy enron moze pomoc w organizacji tej imprezy ?  bylaby to dobra okazja nawiazania wielu pozytecznych kontaktow .  wicek  - - - - - - - - - - - - - - - - - - - - - - forwarded by vince j kaminski / hou / ect on 02 / 17 / 2000  08 : 51 am - - - - - - - - - - - - - - - - - - - - - - - - - - -  " andrzej wodnicki " on 02 / 16 / 2000 02 : 50 : 05 am  to : vince j kaminski / hou / ect @ ect  cc :  subject : aiesec polska - eurolds 2000  szanowny panie kaminski !  nazywam sie andrzej wodnicki i jestem czlonkiem stowarzysznia studentow  aiesec przy szkole glownej handlowej ( dawnej sgpis ) .  prosze o poswiecenie paru chwil na przeczytanie tego maila .  ( kontakt do pana otrzymalem od kolegi , ktory organizowal prezentacje firmy  enron na sgh , a posrednio od pana jarka astramowicza , przedstawiciela enron  na polske . )  w imieniu aiesec polska chcialbym zwrocic sie do pana z wielka prosba pomocy  przy wydarzeniu , ktore w tym rok

In [7]:
id2label = {0: "NOT SPAM", 1: "SPAM"}
label2id = {"NOT SPAM": 0, "SPAM": 1}

**Build and Evaluate Classifier**

In [8]:
def get_sms_messages_string(dataset, item_nums, include_labels=False):
    sms_msgs_string = ""
    for item_num, entry in zip(item_nums, dataset.select(item_nums)):
        sms = entry["Text"]
        label_id = entry["Spam"]

        if include_labels:
            sms_msgs_string +=(
                f"{item_num} (label={id2label[label_id]}) -> {sms}\n\n"
            )

        else:
            sms_msgs_string += f"{item_num} -> {sms}\n\n"
    
    print(sms_msgs_string)

In [9]:
get_sms_messages_string(train, range(1), True)


0 (label=NOT SPAM) -> Subject: aiesec polska - eurolds 2000  jarek ,  czy enron moze pomoc w organizacji tej imprezy ?  bylaby to dobra okazja nawiazania wielu pozytecznych kontaktow .  wicek  - - - - - - - - - - - - - - - - - - - - - - forwarded by vince j kaminski / hou / ect on 02 / 17 / 2000  08 : 51 am - - - - - - - - - - - - - - - - - - - - - - - - - - -  " andrzej wodnicki " on 02 / 16 / 2000 02 : 50 : 05 am  to : vince j kaminski / hou / ect @ ect  cc :  subject : aiesec polska - eurolds 2000  szanowny panie kaminski !  nazywam sie andrzej wodnicki i jestem czlonkiem stowarzysznia studentow  aiesec przy szkole glownej handlowej ( dawnej sgpis ) .  prosze o poswiecenie paru chwil na przeczytanie tego maila .  ( kontakt do pana otrzymalem od kolegi , ktory organizowal prezentacje firmy  enron na sgh , a posrednio od pana jarka astramowicza , przedstawiciela enron  na polske . )  w imieniu aiesec polska chcialbym zwrocic sie do pana z wielka prosba pomocy  przy wydarzeniu , ktore 

In [10]:
sms_msgs_string = get_sms_messages_string(train, range(1,15))

query = f"""
{sms_msgs_string}
---
Classify the messages above as SPAM or NOT SPAM. Respond in JSON format.
Use the following format: {{"0": "NOT SPAM", "1": "SPAM"}}

"""

print(query)

1 -> Subject: vince and stinson ,  i got this resume from my friend ming sit who has a ph . d . from stanford .  please take a look at his resume to see if we can use him . i classify him as  a structurer , but things may change after all these years .  zimin  - - - - - - - - - - - - - - - - - - - - - - forwarded by zimin lu / hou / ect on 05 / 17 / 2000 04 : 08 pm  - - - - - - - - - - - - - - - - - - - - - - - - - - -  " sit , ming " on 05 / 17 / 2000 02 : 41 : 50 pm  to : " zimin lu ( e - mail ) "  cc :  subject :  - resume . doc

2 -> Subject: re : many  helyette ,  sorry for not getting back to you earlier . i took a few days off before my  family goes back to california .  i have arranged a replacement for you for the houston and new york risk  courses . my colleagues , stinson gibner and steve leppard , gave two  presentations . i shall be speaking in london on thursday only . steve leppard  will make a presentation on friday in my place .  i look forward to meeting you in london

In [11]:
response = {
  "1": "NOT SPAM",
  "2": "NOT SPAM",
  "3": "NOT SPAM",
  "4": "NOT SPAM",
  "5": "NOT SPAM",
  "6": "SPAM",
  "7": "NOT SPAM",
  "8": "SPAM",
  "9": "NOT SPAM",
  "10": "SPAM",
  "11": "SPAM",
  "12": "NOT SPAM",
  "13": "NOT SPAM",
  "14": "SPAM"
}

In [12]:
def get_accuracy(response, dataset, og_ind):
    crct = 0
    total = 0

    for entry_num, pred in response.items():
        if int(entry_num) not in og_ind:
            continue

        label_id = dataset[int(entry_num)]["Spam"]
        label = id2label[label_id]

        if pred.lower() == label.lower():
            crct+=1

        total+=1

    accu = crct/total
    return accu

In [13]:
accuracy = get_accuracy(response, train, range(1,15))
accuracy

1.0